# 自编码器对mnist图片降噪

In [1]:
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten
from keras.optimizers import *

D:\study\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 数据预处理

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype('float32') / 255.
X_test = X_train.astype('float32') / 255.

noised_X_train = X_train.copy()

生成用于训练的噪化后的数据

In [4]:
noise_rate = 0.2

for i in range(noised_X_train.shape[0]):
    noised_X_train[i] += noise_rate * np.random.randn(*X_train[0].shape)
    noised_X_train[i] = np.clip(noised_X_train[i], 0.0, 1.0)

noised_X_train.reshape(-1, 1, 28, 28)

# plt.imshow(noised_X_train[1], cmap='Greys')

array([[[[6.78575411e-02, 0.00000000e+00, 3.49057496e-01, ...,
          3.55915606e-01, 6.72059774e-01, 6.72272563e-01],
         [7.65982121e-02, 0.00000000e+00, 9.75382924e-02, ...,
          1.14845246e-01, 1.46436393e-01, 0.00000000e+00],
         [9.22519416e-02, 3.92281592e-01, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         ...,
         [2.83307768e-02, 1.11366376e-01, 0.00000000e+00, ...,
          5.84276132e-02, 1.51957110e-01, 0.00000000e+00],
         [1.52063593e-01, 1.80487141e-01, 4.76990454e-02, ...,
          0.00000000e+00, 2.94646561e-01, 0.00000000e+00],
         [7.28144169e-01, 8.45670477e-02, 0.00000000e+00, ...,
          3.68184030e-01, 0.00000000e+00, 1.87570810e-01]]],


       [[[9.21213999e-02, 1.06455408e-01, 5.29946804e-01, ...,
          4.80639070e-01, 2.16796976e-02, 3.88998449e-01],
         [1.18623123e-01, 2.60325342e-01, 1.03372693e-01, ...,
          1.80349369e-02, 9.50510874e-02, 0.00000000e+00],
      

In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2])

In [6]:
noised_X_train.shape

(60000, 28, 28)

## 建模

In [7]:
model = Sequential()
model.add(Convolution2D(32, 5, 5, 
                       border_mode='same',
                       input_shape=(1, 28, 28)))

model.add(MaxPooling2D( pool_size=(2,2),   #每次处理 2X2 的方格
                        strides=(2,2),    # 这个 2X2 的方格移动的步长
                        border_mode='same'))

model.add(Convolution2D(64, 5, 5, border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2), border_mode='same', strides=(2,2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(784))

model.compile(optimizer=Adam(lr=0.01), loss='MSE', metrics=['accuracy'])

model.fit(noised_X_train, X_train, epochs=2)

D:\study\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(1, 28, 28..., padding="same")`
  after removing the cwd from sys.path.
D:\study\Anaconda\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
  
D:\study\Anaconda\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
D:\study\Anaconda\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same")`
  # This is added back by InteractiveShellApp.init_path()


ValueError: Error when checking input: expected conv2d_1_input to have 4 dimensions, but got array with shape (60000, 28, 28)